# 모델을 만들기 위한 절차

1. 2개 이상의 데이터가 들어오면 자동으로 처리할 수 있게 해주는 모델 개발
2. 현재는 4개의 데이터를 직접 전처리해 한개의 데이터를 추출할 수 있게 해놓음
3. 기온과 습도 풍향과 풍속은 그날 바로 알 수 있으므로, 이 4개 데이터를 넣으면 그 지역 현재 시간대의 미세먼지 등급을 알 수 있게 하는 모델

결론적으로 처리할 건  
(이렇게 하려면 실시간으로 데이터를 크롤링 해와 데이터를 쉽게 다룰 수 있도록 바꾸는 과정이 필요함. -> 현재 불가)  
(그냥 과거 데이터를 그냥 두고, 입력을 4개하면 그 지역 미세먼지 등급을 예측하는 모델을 생각해보자.)  
input : 입력하세요 -> 기온, 습도, 풍향, 풍속, 위도(거의 비슷한 위치이므로 생략), 경도(거의 비슷한 위치이므로 생략)  
처리중입니다. -> 결과 : **지역 미세먼지 등급 - 좋음



다음에 해야할 작업 - >
1. 위도, 경도 데이터를 대안동을 뺀 나머지 데이터에 다 넣어주고,
2. 위도, 경도, 풍향, 풍속, 기온, 습도 6가지 데이터의 특징을 찾아보고 (없을 확률이 높다)  

-> 한개지역의 데이터씩 뽑아서 <올라가거나 내려가는 구간>에서는 회귀분석을 통해, 나온 예측값들을 조화평균내서 대안동에 대입하고, 거의 일정한 구간에서는 그 값들의 평균을 내 대안동에 대입해서 pm25를 예측하는 알고리즘을 만들어서 만들고, 그 값들을 SVM, KNN 등등에 대입해 4가지 등급을 예측하는 알고리즘을 만들자 


-> 올라가거나 내려가는 구간을 판별하는 방식은 경사하강법? 미분을 적용시켜 속도가 변하는 구간이 발생하는 조건을 넣어서 생기는 구간을 만들면 되지 않을까 생각한다.

## 해야할 목표
1. 나쁨과 보통의 기준을 나눠 아림 데이터의 나쁨을 잘 나타낼 수 있는 방향성을 찾기 16 ~ 35,36 ~ 75
    - 나쁨과 보통 기준 데이터만 모아서 아림과 airKorea의 데이터 차이를 한번 비교해보기 ( 일단 해봐야 알 것 같다 )
2. PM2.5를 예측할 수 있게 데이터를 조작해보기 ( 이것도 일단 해봐야 알 것 같다 )

In [1]:
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
from datetime import timedelta, datetime
from matplotlib import font_manager, rc

from IPython.display import Image # 이미지 삽입

plt.rcParams['font.family'] = 'Malgun Gothic'
plt.rcParams['figure.figsize'] = [25,15]
plt.rcParams['font.size'] = 25

In [2]:
# outliar 제거 함수 (IQR 설정) (1)
def remove_out(dataframe, remove_col) :
    dff = dataframe
    for k in remove_col :
        level_1q = dff[k].quantile(0.25)
        level_3q = dff[k].quantile(0.75)
        IQR = level_3q - level_1q
        rev_range = 1.5
        outliar_h = dff[k] >= level_3q + (rev_range * IQR)
        outliar_l = dff[k] <= level_1q - (rev_range * IQR)
        a=dff[outliar_h].index
        b=dff[outliar_l].index
        dff.drop(a, inplace=True)
        dff.drop(b, inplace=True)

#그래프 2개 비교 (차이 비교) -- 숫자를 표시안함
def difference(dataframe1, dataframe2) :    
    diff = np.subtract(dataframe1,dataframe2)
    
    # diff가 0보다 작은 index 찾는 것
    diff_down = dataframe2.loc[diff.loc[diff < 0].index]
    display(diff_down.index)
    
    # diff의 값을 평균내는 부분
    print(diff.describe())
    
    # 20%인 위치 값 찾기
    try :
        sorted_diff = diff.sort_values()
        ten_percent_index = int(len(sorted_diff) * 0.1)
        ten_percent_value = sorted_diff.iloc[ten_percent_index]
        print("10% value: ", ten_percent_value)
    except IndexError as e:
        if "single positional indexer is out-of-bounds" in str(e):
            print("매우나쁨 농도가 없습니다.")
        else:
            raise e
    finally :
        # 숫자 넣는 부분, height + 0.25로 숫자 약간 위로 위치하게 조정
        plt.plot(dataframe1.index,diff)
        for i in range(len(dataframe1.index)):
            height = diff[i]
            plt.text(dataframe1.index[i], height + 0.01, '%.2f' %height, ha='center', va='bottom', size = 25)
        plt.title("Difference between airm-airKorea")
        plt.xlabel("Hour")
        plt.ylabel("diff")
        plt.show()
    
# 아림의 미세먼지 농도가 매우나쁨 이상일 때, airKorea의 미세먼지 농도의 평가
def air_quality_label(pm25):
    if pm25 <= 15:
        return '좋음'
    elif pm25 <= 35:
        return '보통'
    elif pm25 <= 75:
        return '나쁨'
    else:
        return '매우 나쁨'